In [7]:
"""
============================================================================
02_fairness_analysis_and_calibration.py
============================================================================
Purpose: Comprehensive fairness analysis and bias mitigation for TBI model
- Loadthe test set 
- Detect bias across ALL demographics (age, sex, employment, income, education)
- Calibrate ONLY variables with significant bias (p < 0.05)
- Bootstrap comparison (5,000 iterations) to assess calibration impact
- Generate visualizations and reports

Author: saumya sharma
Date: 24 dec 2025
Input: 
  - output/models/best_model_final.pkl (trained model)
  - data/processed/df17nov.csv (full dataset)
Output: 
  - Bias detection results
  - Calibrated predictions
  - Bootstrap comparison results
  - Fairness visualizations
============================================================================
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.utils import resample
from scipy.stats import ttest_ind, f_oneway

# Set random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Create output directories
os.makedirs("output/fairness", exist_ok=True)
os.makedirs("output/fairness/figures", exist_ok=True)
os.makedirs("output/fairness/tables", exist_ok=True)

print("="*80)
print("FAIRNESS ANALYSIS AND BIAS MITIGATION PIPELINE")
print("="*80)

# ============================================================================
# 1. LOAD MODEL AND RECREATE TEST SET
# ============================================================================
print("\n" + "="*80)
print("STEP 1: Loading Model and Test Set")
print("="*80)

# Load model
best_model_path = "output/models/best_model_final.pkl"
best_model_instance = joblib.load(best_model_path)
print(f"✓ Loaded model from {best_model_path}")

# Load full dataset
filepath = "../data/processed/df17nov.csv"
df = pd.read_csv(filepath)
print(f"✓ Loaded dataset: {df.shape[0]} rows × {df.shape[1]} columns")

# Separate features and target
target_col = 'FIM_change'
X_full = df.drop(columns=[target_col, 'Mod1Id'], errors='ignore')
y_full = df[target_col]

# CRITICAL: Recreate the EXACT same train/test split from modeling pipeline
# This ensures we only use TEST SET for fairness analysis
X_train, X_test, y_train, y_test = train_test_split(
    X_full, y_full, 
    test_size=0.2, 
    random_state=RANDOM_SEED, 
    shuffle=True
)

print(f"\n✓ Test set size: {X_test.shape[0]} samples")
print(f"  (Training set: {X_train.shape[0]} samples - NOT used for fairness analysis)")

# Ensure feature alignment with model
model_features = best_model_instance.feature_names_in_
X_test = X_test[model_features]

# Create test dataframe for analysis
df_test = X_test.copy()
df_test['FIM_change'] = y_test.values

# ============================================================================
# 2. DEFINE DEMOGRAPHIC SUBGROUPS
# ============================================================================
print("\n" + "="*80)
print("STEP 2: Defining Demographic Subgroups")
print("="*80)

# Age groups
age_bins = [0, 30, 50, 100]
age_labels = ['<30', '30-50', '50+']
df_test['AgeGroup'] = pd.cut(df_test['AGENoPHI'], bins=age_bins, labels=age_labels)
print(f"✓ Age groups: {age_labels}")

# Sex
df_test['Sex'] = df_test['SexF'].map({1: 'Female', 2: 'Male'})
print(f"✓ Sex: Female, Male")

# Employment (simplified to avoid too many small categories)
emp_map = {
    5: 'Employed',
    2: 'Student',
    3: 'Student',
    9: 'Retired',
    12: 'Retired',
    15: 'Retired',
    10: 'Unemployed',
    13: 'Unemployed',
}
df_test['Employment'] = df_test['Emp1'].map(emp_map).fillna('Other')
print(f"✓ Employment: Employed, Student, Retired, Unemployed, Other")

# Income (grouped into broader categories)
def categorize_income(earn_code):
    if pd.isna(earn_code) or earn_code in [666, 777, 888, 999]:
        return 'Unknown'
    elif earn_code in [1, 2]:
        return '<$20k'
    elif earn_code in [3, 4, 5]:
        return '$20k-$50k'
    elif earn_code in [6, 7, 8, 9, 10]:
        return '$50k-$100k'
    elif earn_code == 11:
        return '$100k+'
    else:
        return 'Unknown'

df_test['Income'] = df_test['Earn'].apply(categorize_income)
print(f"✓ Income: <$20k, $20k-$50k, $50k-$100k, $100k+, Unknown")

# Education (grouped)
def categorize_education(edu_years):
    if pd.isna(edu_years) or edu_years in [666, 999]:
        return 'Unknown'
    elif edu_years <= 12:
        return 'High School or Less'
    elif edu_years in [13, 14]:
        return 'Associate'
    elif edu_years in [15, 16]:
        return 'Bachelors'
    elif edu_years >= 17:
        return 'Graduate'
    else:
        return 'Unknown'

df_test['Education'] = df_test['EduYears'].apply(categorize_education)
print(f"✓ Education: High School or Less, Associate, Bachelors, Graduate, Unknown")

# Subgroup dictionary
subgroups = {
    'AgeGroup': df_test['AgeGroup'],
    'Sex': df_test['Sex'],
    'Employment': df_test['Employment'],
    'Income': df_test['Income'],
    'Education': df_test['Education']
}

print(f"\n✓ Total subgroups defined: {len(subgroups)}")

# ============================================================================
# 3. GENERATE ORIGINAL PREDICTIONS
# ============================================================================
print("\n" + "="*80)
print("STEP 3: Generating Original Predictions")
print("="*80)

y_pred_original = best_model_instance.predict(X_test)
residuals_original = y_pred_original - y_test.values

df_test['y_pred_original'] = y_pred_original
df_test['Residual_original'] = residuals_original

print(f"✓ Predictions generated for {len(y_pred_original)} test samples")
print(f"  Mean residual: {residuals_original.mean():.3f}")
print(f"  Std residual: {residuals_original.std():.3f}")

# ============================================================================
# 4. BIAS DETECTION: ANOVA/T-TEST FOR ALL DEMOGRAPHICS
# ============================================================================
print("\n" + "="*80)
print("STEP 4: Bias Detection (ANOVA/T-Test)")
print("="*80)

bias_results = []
variables_to_calibrate = []

for grp_name, grp_col in subgroups.items():
    print(f"\n--- Testing {grp_name} ---")
    
    # Get unique categories (drop NaN)
    categories = grp_col.dropna().unique()
    
    # Skip if only 1 category
    if len(categories) < 2:
        print(f"  ⚠ Skipped: Only {len(categories)} category")
        continue
    
    # Collect residuals by category
    groups = [residuals_original[grp_col == cat] for cat in categories]
    
    # Filter out empty groups
    groups = [g for g in groups if len(g) > 0]
    
    if len(groups) < 2:
        print(f"  ⚠ Skipped: Not enough non-empty groups")
        continue
    
    # Statistical test
    if len(groups) == 2:
        # Binary variable: t-test
        t_stat, p_val = ttest_ind(groups[0], groups[1], equal_var=False)
        test_stat = t_stat
        test_name = "t-test"
        print(f"  t-test: t = {t_stat:.3f}, p = {p_val:.4f}")
    else:
        # Multi-category: ANOVA
        f_stat, p_val = f_oneway(*groups)
        test_stat = f_stat
        test_name = "ANOVA"
        print(f"  ANOVA: F = {f_stat:.3f}, p = {p_val:.4f}")
    
    # Record results
    bias_results.append({
        'Variable': grp_name,
        'Test': test_name,
        'Statistic': test_stat,
        'p_value': p_val,
        'Significant': 'Yes' if p_val < 0.05 else 'No'
    })
    
    # Mark for calibration if significant
    if p_val < 0.05:
        variables_to_calibrate.append(grp_name)
        print(f"  ✓ SIGNIFICANT BIAS DETECTED (p = {p_val:.4f})")
    else:
        print(f"  ✓ No significant bias (p = {p_val:.4f})")

# Save bias detection results
bias_df = pd.DataFrame(bias_results)
bias_df.to_csv("output/fairness/tables/bias_detection_results.csv", index=False)

print("\n" + "="*80)
print("BIAS DETECTION SUMMARY")
print("="*80)
print(bias_df.to_string(index=False))

if len(variables_to_calibrate) > 0:
    print(f"\n⚠ Variables requiring calibration: {', '.join(variables_to_calibrate)}")
else:
    print("\n✓ No variables require calibration (no significant bias detected)")

# ============================================================================
# 5. POST-HOC CALIBRATION (ONLY FOR BIASED VARIABLES)
# ============================================================================
print("\n" + "="*80)
print("STEP 5: Post-Hoc Calibration")
print("="*80)

# Start with original predictions
df_test['y_pred_calibrated'] = df_test['y_pred_original'].copy()

calibration_applied = {}

if len(variables_to_calibrate) == 0:
    print("✓ No calibration needed (no significant bias detected)")
else:
    for var_name in variables_to_calibrate:
        print(f"\n--- Calibrating {var_name} ---")
        
        grp_col = subgroups[var_name]
        
        # Compute mean residual per category
        bias_by_group = df_test.groupby(grp_col)['Residual_original'].mean()
        
        print("  Mean residuals before calibration:")
        for cat, bias_val in bias_by_group.items():
            print(f"    {cat}: {bias_val:.3f}")
        
        # Apply calibration: subtract group bias
        def calibrate_prediction(row):
            if pd.isna(row[var_name]):
                return row['y_pred_calibrated']  # Don't calibrate missing
            else:
                return row['y_pred_calibrated'] - bias_by_group[row[var_name]]
        
        df_test['y_pred_calibrated'] = df_test.apply(
            lambda row: calibrate_prediction(row), axis=1
        )
        
        # Store calibration info
        calibration_applied[var_name] = bias_by_group.to_dict()
        
        print(f"  ✓ Calibration applied for {var_name}")

# Recalculate residuals after calibration
df_test['Residual_calibrated'] = df_test['y_pred_calibrated'] - y_test.values

print(f"\n✓ Calibration complete")
print(f"  Original mean residual: {df_test['Residual_original'].mean():.3f}")
print(f"  Calibrated mean residual: {df_test['Residual_calibrated'].mean():.3f}")

# ============================================================================
# 6. RE-RUN ANOVA TO VERIFY BIAS REDUCTION
# ============================================================================
print("\n" + "="*80)
print("STEP 6: Verification (Re-run ANOVA After Calibration)")
print("="*80)

if len(variables_to_calibrate) == 0:
    print("✓ Skipped (no calibration was applied)")
else:
    verification_results = []
    
    for var_name in variables_to_calibrate:
        print(f"\n--- Verifying {var_name} ---")
        
        grp_col = subgroups[var_name]
        categories = grp_col.dropna().unique()
        
        # Original residuals
        groups_orig = [df_test[grp_col == cat]['Residual_original'] for cat in categories]
        groups_orig = [g for g in groups_orig if len(g) > 0]
        
        # Calibrated residuals
        groups_cal = [df_test[grp_col == cat]['Residual_calibrated'] for cat in categories]
        groups_cal = [g for g in groups_cal if len(g) > 0]
        
        # ANOVA
        if len(groups_orig) >= 2:
            f_orig, p_orig = f_oneway(*groups_orig)
            f_cal, p_cal = f_oneway(*groups_cal)
            
            print(f"  Before: F = {f_orig:.3f}, p = {p_orig:.4f}")
            print(f"  After:  F = {f_cal:.3f}, p = {p_cal:.4f}")
            
            if p_cal >= 0.05:
                print(f"  ✓ Bias successfully eliminated (p = {p_cal:.4f} >= 0.05)")
            else:
                print(f"  ⚠ Residual bias remains (p = {p_cal:.4f} < 0.05)")
            
            verification_results.append({
                'Variable': var_name,
                'F_before': f_orig,
                'p_before': p_orig,
                'F_after': f_cal,
                'p_after': p_cal,
                'Bias_Eliminated': 'Yes' if p_cal >= 0.05 else 'No'
            })
    
    # Save verification results
    if verification_results:
        verification_df = pd.DataFrame(verification_results)
        verification_df.to_csv("output/fairness/tables/calibration_verification.csv", index=False)
        print("\n" + "="*80)
        print("VERIFICATION SUMMARY")
        print("="*80)
        print(verification_df.to_string(index=False))

# ============================================================================
# 7. BOOTSTRAP COMPARISON (5,000 ITERATIONS)
# ============================================================================
print("\n" + "="*80)
print("STEP 7: Bootstrap Comparison (n=5,000)")
print("="*80)

y_true = y_test.values
y_pred_orig = df_test['y_pred_original'].values
y_pred_cal = df_test['y_pred_calibrated'].values

N_BOOTSTRAP = 5000
mae_diffs = []
r2_diffs = []

print(f"Running {N_BOOTSTRAP} bootstrap iterations...")

for i in range(N_BOOTSTRAP):
    if (i + 1) % 1000 == 0:
        print(f"  Completed {i + 1}/{N_BOOTSTRAP} iterations...")
    
    # Resample with replacement
    idx = resample(np.arange(len(y_true)), replace=True, random_state=RANDOM_SEED + i)
    
    y_boot = y_true[idx]
    y_orig_boot = y_pred_orig[idx]
    y_cal_boot = y_pred_cal[idx]
    
    # Compute metrics
    mae_orig = mean_absolute_error(y_boot, y_orig_boot)
    mae_cal = mean_absolute_error(y_boot, y_cal_boot)
    r2_orig = r2_score(y_boot, y_orig_boot)
    r2_cal = r2_score(y_boot, y_cal_boot)
    
    # Store differences (Calibrated - Original)
    mae_diffs.append(mae_cal - mae_orig)
    r2_diffs.append(r2_cal - r2_orig)

# Compute statistics
mae_diff_mean = np.mean(mae_diffs)
r2_diff_mean = np.mean(r2_diffs)

mae_ci = (np.percentile(mae_diffs, 2.5), np.percentile(mae_diffs, 97.5))
r2_ci = (np.percentile(r2_diffs, 2.5), np.percentile(r2_diffs, 97.5))

print("\n" + "="*80)
print("BOOTSTRAP RESULTS")
print("="*80)
print(f"\nMAE Difference (Calibrated - Original):")
print(f"  Mean: {mae_diff_mean:.4f}")
print(f"  95% CI: [{mae_ci[0]:.4f}, {mae_ci[1]:.4f}]")
print(f"  Interpretation: {'No significant change' if mae_ci[0] < 0 < mae_ci[1] else 'Significant change'}")

print(f"\nR² Difference (Calibrated - Original):")
print(f"  Mean: {r2_diff_mean:.4f}")
print(f"  95% CI: [{r2_ci[0]:.4f}, {r2_ci[1]:.4f}]")
print(f"  Interpretation: {'No significant change' if r2_ci[0] < 0 < r2_ci[1] else 'Significant change'}")

# Save bootstrap results
bootstrap_results = pd.DataFrame({
    'Metric': ['MAE_diff', 'R2_diff'],
    'Mean': [mae_diff_mean, r2_diff_mean],
    'CI_lower': [mae_ci[0], r2_ci[0]],
    'CI_upper': [mae_ci[1], r2_ci[1]],
    'Significant': [
        'No' if mae_ci[0] < 0 < mae_ci[1] else 'Yes',
        'No' if r2_ci[0] < 0 < r2_ci[1] else 'Yes'
    ]
})
bootstrap_results.to_csv("output/fairness/tables/bootstrap_comparison.csv", index=False)

# ============================================================================
# 8. COMPUTE METRICS PER SUBGROUP
# ============================================================================
print("\n" + "="*80)
print("STEP 8: Computing Metrics Per Subgroup")
print("="*80)

metrics_list = []

for grp_name, grp_col in subgroups.items():
    for category in grp_col.unique():
        if pd.isna(category):
            continue
        
        idx = grp_col == category
        
        if idx.sum() == 0:
            continue
        
        y_true_grp = y_test.values[idx]
        y_pred_orig_grp = y_pred_orig[idx]
        y_pred_cal_grp = y_pred_cal[idx]
        
        metrics_list.append({
            'Subgroup': grp_name,
            'Category': category,
            'N': len(y_true_grp),
            'MAE_original': mean_absolute_error(y_true_grp, y_pred_orig_grp),
            'MAE_calibrated': mean_absolute_error(y_true_grp, y_pred_cal_grp),
            'R2_original': r2_score(y_true_grp, y_pred_orig_grp),
            'R2_calibrated': r2_score(y_true_grp, y_pred_cal_grp),
            'MeanResidual_original': (y_pred_orig_grp - y_true_grp).mean(),
            'MeanResidual_calibrated': (y_pred_cal_grp - y_true_grp).mean()
        })

metrics_df = pd.DataFrame(metrics_list)
metrics_df.to_csv("output/fairness/tables/subgroup_metrics.csv", index=False)

print(f"✓ Computed metrics for {len(metrics_df)} subgroups")
print("\nSample of subgroup metrics:")
print(metrics_df.head(10).to_string(index=False))

# ============================================================================
# 9. VISUALIZATIONS
# ============================================================================
print("\n" + "="*80)
print("STEP 9: Generating Visualizations")
print("="*80)

sns.set(style="whitegrid", palette="tab20", font_scale=1.1)

for grp_name in subgroups.keys():
    print(f"\n  Creating plots for {grp_name}...")
    
    df_grp = metrics_df[metrics_df['Subgroup'] == grp_name]
    
    # --- MEAN RESIDUALS: BEFORE VS AFTER CALIBRATION ---
    fig, ax = plt.subplots(figsize=(10, 6))
    
    x = np.arange(len(df_grp))
    width = 0.35
    
    ax.bar(x - width/2, df_grp['MeanResidual_original'], width, 
           label='Original', color='coral', alpha=0.8)
    ax.bar(x + width/2, df_grp['MeanResidual_calibrated'], width,
           label='Calibrated', color='skyblue', alpha=0.8)
    
    ax.set_xlabel(grp_name, fontsize=12)
    ax.set_ylabel('Mean Residual (Pred - True)', fontsize=12)
    ax.set_title(f'Bias Reduction: {grp_name}', fontsize=14)
    ax.set_xticks(x)
    ax.set_xticklabels(df_grp['Category'], rotation=45, ha='right')
    ax.axhline(0, color='red', linestyle='--', linewidth=1)
    ax.legend()
    ax.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f"output/fairness/figures/bias_comparison_{grp_name}.png", 
                dpi=300, bbox_inches='tight')
    plt.close()

print("\n✓ All visualizations saved")

# ============================================================================
# 10. SAVE FINAL OUTPUTS
# ============================================================================
print("\n" + "="*80)
print("STEP 10: Saving Final Outputs")
print("="*80)

# Save predictions
predictions_output = df_test[[
    'y_pred_original', 
    'y_pred_calibrated', 
    'Residual_original', 
    'Residual_calibrated'
]].copy()
predictions_output['y_true'] = y_test.values
predictions_output.to_csv("output/fairness/tables/predictions_comparison.csv", index=False)
print("✓ Saved predictions comparison")

# Save calibration adjustments
if calibration_applied:
    cal_df_list = []
    for var_name, bias_dict in calibration_applied.items():
        for category, bias_value in bias_dict.items():
            cal_df_list.append({
                'Variable': var_name,
                'Category': category,
                'Bias_Correction': bias_value
            })
    cal_df = pd.DataFrame(cal_df_list)
    cal_df.to_csv("output/fairness/tables/calibration_adjustments.csv", index=False)
    print("✓ Saved calibration adjustments")

# ============================================================================
# 11. FINAL SUMMARY
# ============================================================================
print("\n" + "="*80)
print("FAIRNESS ANALYSIS COMPLETE")
print("="*80)

print("\n📊 Summary:")
print(f"  • Test set size: {len(y_test)}")
print(f"  • Demographics analyzed: {len(subgroups)}")
print(f"  • Variables with significant bias: {len(variables_to_calibrate)}")
print(f"  • Calibration applied to: {', '.join(variables_to_calibrate) if variables_to_calibrate else 'None'}")
print(f"  • Bootstrap iterations: {N_BOOTSTRAP}")

print("\n📁 Output files saved to:")
print("  • output/fairness/tables/")
print("    - bias_detection_results.csv")
print("    - calibration_verification.csv")
print("    - bootstrap_comparison.csv")
print("    - subgroup_metrics.csv")
print("    - predictions_comparison.csv")
print("    - calibration_adjustments.csv")
print("  • output/fairness/figures/")
print("    - bias_comparison_[variable].png (for each demographic)")

print("\n" + "="*80)
print("✅ PIPELINE EXECUTION COMPLETE")
print("="*80)

FAIRNESS ANALYSIS AND BIAS MITIGATION PIPELINE

STEP 1: Loading Model and Test Set
✓ Loaded model from output/models/best_model_final.pkl
✓ Loaded dataset: 13704 rows × 137 columns

✓ Test set size: 2741 samples
  (Training set: 10963 samples - NOT used for fairness analysis)

STEP 2: Defining Demographic Subgroups
✓ Age groups: ['<30', '30-50', '50+']
✓ Sex: Female, Male
✓ Employment: Employed, Student, Retired, Unemployed, Other
✓ Income: <$20k, $20k-$50k, $50k-$100k, $100k+, Unknown
✓ Education: High School or Less, Associate, Bachelors, Graduate, Unknown

✓ Total subgroups defined: 5

STEP 3: Generating Original Predictions
✓ Predictions generated for 2741 test samples
  Mean residual: -0.321
  Std residual: 14.824

STEP 4: Bias Detection (ANOVA/T-Test)

--- Testing AgeGroup ---
  ANOVA: F = 4.363, p = 0.0128
  ✓ SIGNIFICANT BIAS DETECTED (p = 0.0128)

--- Testing Sex ---
  t-test: t = 0.670, p = 0.5030
  ✓ No significant bias (p = 0.5030)

--- Testing Employment ---
  ANOVA: F = 3

C:\Users\saumy\AppData\Local\Temp\ipykernel_17564\3116176886.py:282: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  bias_by_group = df_test.groupby(grp_col)['Residual_original'].mean()


  Completed 1000/5000 iterations...
  Completed 2000/5000 iterations...
  Completed 3000/5000 iterations...
  Completed 4000/5000 iterations...
  Completed 5000/5000 iterations...

BOOTSTRAP RESULTS

MAE Difference (Calibrated - Original):
  Mean: -0.0408
  95% CI: [-0.1000, 0.0213]
  Interpretation: No significant change

R² Difference (Calibrated - Original):
  Mean: 0.0039
  95% CI: [-0.0029, 0.0107]
  Interpretation: No significant change

STEP 8: Computing Metrics Per Subgroup
✓ Computed metrics for 20 subgroups

Sample of subgroup metrics:
  Subgroup   Category    N  MAE_original  MAE_calibrated  R2_original  R2_calibrated  MeanResidual_original  MeanResidual_calibrated
  AgeGroup        50+  788     11.461699       11.463366     0.220628       0.221406              -1.418887                 1.006101
  AgeGroup      30-50  811     10.761700       10.698086     0.309174       0.314892               0.685186                -0.013302
  AgeGroup        <30 1041     10.076342       10